# 线性回归-从0开始

## 线性回归

In [15]:
# 创建数据集
from mxnet import ndarray as nd
from mxnet import autograd

num_inputs = 2
num_examples = 1000

true_w = [2, -3.4]
true_b = 4.2

X = nd.random_normal(shape=(num_examples, num_inputs))
y = true_w[0] * X[:, 0] + true_w[1] * X[:, 1] + true_b
y += .01 * nd.random_normal(shape=y.shape)
print(X[0], y[0])

(
[ 0.75390846 -1.08878255]
<NDArray 2 @cpu(0)>, 
[ 9.39740658]
<NDArray 1 @cpu(0)>)


In [16]:
# 数据读取
import random
batch_size = 10
def data_iter():
    # 产生一个随机索引
    idx = list(range(num_examples))
    random.shuffle(idx)
    for i in range(0, num_examples, batch_size):
        j = nd.array(idx[i:min(i+batch_size, num_examples)])
        yield nd.take(X, j), nd.take(y, j)

In [17]:
for data, label in data_iter():
    print(data, label)
    break

(
[[ 1.1323204   0.14613174]
 [ 0.08220391 -0.90140879]
 [ 0.02123423  1.01816428]
 [-1.010396   -0.82747835]
 [ 0.20307845 -0.43065679]
 [ 0.6785391  -0.43320397]
 [ 1.04204953 -0.34072676]
 [-0.33241424  0.35802266]
 [-0.41049707  0.73333913]
 [-1.74850154 -1.66190219]]
<NDArray 10x2 @cpu(0)>, 
[ 5.98380661  7.44668674  0.79245573  4.98291016  6.05733633  7.03368902
  7.42522478  2.30050564  0.88386679  6.3548584 ]
<NDArray 10 @cpu(0)>)


In [18]:
# 初始化模型参数
w = nd.random_normal(shape=(num_inputs, 1))
b = nd.zeros((1,))
params = [w, b]

for param in params:
    param.attach_grad()

In [19]:
# 定义模型
def net(X):
    return nd.dot(X, w) + b
# 损失函数
def square_loss(yhat, y):
    return (yhat - y.reshape(yhat.shape)) ** 2

In [20]:
# 优化
def SGD(params, lr):
    for param in params:
        param[:] = param - lr * param.grad

In [21]:
# 训练
epochs = 5
learning_rate = .001
for e in range(epochs):
    total_loss = 0
    for data, label in data_iter():
        with autograd.record():
            output = net(data)
            loss = square_loss(output, label)
        loss.backward()
        SGD(params, learning_rate)
            
        total_loss += nd.sum(loss).asscalar()
    print("Epoch %d, average loss: %f" % (e, total_loss/num_examples))

Epoch 0, average loss: 9.416063
Epoch 1, average loss: 0.165376
Epoch 2, average loss: 0.003059
Epoch 3, average loss: 0.000154
Epoch 4, average loss: 0.000100


In [22]:
print true_w, w
print true_b, b

[2, -3.4] 
[[ 2.00055385]
 [-3.39974689]]
<NDArray 2x1 @cpu(0)>
4.2 
[ 4.19946671]
<NDArray 1 @cpu(0)>
